In [53]:
import csv
import numpy as np
import pandas as pd
import os
import re


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.feature_selection import SelectFromModel

### First Data

In [71]:
data = pd.read_csv("labels_sessions.csv")
lab_data = data[data.Device == "A010C3"]
daily_data = data[data.KnownLabel != "NaN"]
foo = data.dropna()
pd.set_option('display.max_rows', 1000)
foo

,Device,Lab_sub,Mean,Peak,Perc20,Perc80,QuartDev,SessionID,TwoClassLabel,ThreeClassLabel,KnownLabel,Start_time,Std,Strong_Peak,dataQual,endTime
5,A010C3,2,0.236833,1.666667,0.222486,0.246834,0.017398,A010C3,H,H,H,1.552992e+09,0.014509,0.000000,0.263162,1.552992e+09
6,A010C3,2,0.253660,1.666667,0.240656,0.270375,0.027688,A010C3,H,H,H,1.552992e+09,0.022901,0.000000,0.301050,1.552992e+09
7,A010C3,2,0.256827,1.666667,0.235741,0.278161,0.034007,A010C3,H,H,H,1.552992e+09,0.023937,0.000000,0.300263,1.552992e+09
8,A010C3,2,0.245208,1.666667,0.228998,0.262336,0.025854,A010C3,H,H,H,1.552992e+09,0.018656,0.000000,0.278081,1.552992e+09
9,A010C3,2,0.264680,0.833333,0.255173,0.275594,0.016077,A010C3,H,H,H,1.552992e+09,0.011808,0.000000,0.280651,1.552992e+09
10,A010C3,2,0.110164,0.000000,0.044698,0.267397,0.169491,A010C3,H,H,H,1.552992e+09,0.184604,0.000000,0.283965,1.552992e+09
11,A010C3,2,0.132290,5.000000,0.077349,0.310424,0.160772,A010C3,H,H,H,1.552992e+09,0.238553,0.000000,0.541605,1.552992e+09
12,A010C3,2,0.600413,11.666667,0.180244,0.902719,0.618759,A010C3,H,H,H,1.552992e+09,0.379870,0.000000,1.531574,1.552992e+09
13,A010C3,2,1.347137,13.333333,0.827341,1.890101,0.951704,A010C3,H,H,H,1.552992e+09,0.469573,0.000000,2.035870,1.552992e+09
14,A010C3,2,2.177368,15.000000,1.706667,2.732669,0.897285,A010C3,H,H,H,1.552992e+09,0.473311,0.000000,2.888666,1.552992e+09


In [58]:
T1 = data[data.ThreeClassLabel == "L"]
print(len(T1))
T2 = data[data.ThreeClassLabel == "M"]
print(len(T2))
T3 = data[data.ThreeClassLabel == "H"]
print(len(T3))

35307
80652
13885


In [59]:
print(len(lab_data))
print(len(daily_data))

562
129282


### Logistic Regression

#### Feature Selection

In [62]:
# Features and target
X = daily_data[['Mean', 'Peak', 'Perc20', 'Perc80', 'QuartDev','Std', 'Strong_Peak', 'dataQual']]
y = daily_data['TwoClassLabel']

X_train = lab_data[['Mean', 'Peak', 'Perc20', 'Perc80', 'QuartDev','Std', 'Strong_Peak', 'dataQual']]
y_train = lab_data['TwoClassLabel']

X_test = daily_data[['Mean', 'Peak', 'Perc20', 'Perc80', 'QuartDev','Std', 'Strong_Peak', 'dataQual']]
y_test = daily_data['TwoClassLabel']

Since Logistic Regression balances the calsses inside of its own by default, no extra upclassing is needed.

In [63]:
from sklearn.linear_model import LogisticRegressionCV

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = LogisticRegressionCV(cv=10, multi_class='ovr', class_weight='balanced').fit(X_train, y_train)
clf.predict(X_test)
clf.score(X_test, y_test)

0.5225277487720926